<div style="background-image:url(images/bertinoro.jpg); padding: 10px 30px 20px 30px; background-size:cover; background-opacity:50%; border-radius:5px">
<p style="float:right; margin-top:20px; padding: 20px 20px 0px 20px; background:rgba(255,255,255,0.6); border-radius:10px;">
<img width="400px" src=images/obspy_logo_full_524x179px.png?raw=true>
</p>

<h1 style="color:#333">First TIDES Training School</h1>
<h5 style="color:#FFF">Bertinoro (FC), Italy, June 1-5 2015</h5>

<h3 style="color:#EEE">Day 1: Data Acquisition and Processing with ObsPy</h3>

<h2 style="color:#FFF">Basic Downloading/Processing Exercise</h2>
</div>

For the this exercise we will download some data from the Tohoku-Oki earthquake, cut out a certain time window around the first arrival and remove the instrument response from the data.

In [ ]:
%pylab inline
from __future__ import print_function
import matplotlib.pylab as plt
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = 12, 8

The first step is to download all the necessary information using the ObsPy FDSN client.

If that does not work due to network constraints, please use the following files (which have the same content):

```
data/tohoku.xml
data/IV_BOB.xml
data/IV_BOB.mseed
```

In [ ]:
import obspy
from obspy.fdsn import Client

c = Client("INGV")

# Event time.
event_time = obspy.UTCDateTime("2011-03-11T05:46:23.2")

# Get the event information. The temporal and magnitude constraints make it unique
cat = c.get_events(starttime=event_time - 10, endtime=event_time + 10,
                   minmagnitude=9)
print(cat)

# Download station information at the response level!
inv = c.get_stations(network="IV", station="BOB", location="", channel="BH?",
                     starttime=event_time - 60, endtime=event_time + 3600,
                     level="response")
print(inv)

# Download 3 component waveforms.
st = c.get_waveforms(network="IV", station="BOB", location="",
                     channel="BH?", starttime=event_time - 60,
                     endtime=event_time + 3600)
print(st)

Have a look at the just downloaded data.

## Exercise

The goal of this exercise is to cut the data from 1 minute before the first arrival to 5 minutes after it, and then remove the instrument response.


#### Step 1: Determine Coordinates of Station

#### Step 2: Determine Coordinates of Event

#### Step 3: Calculate distance of event and station.

Use `obspy.core.util.geodetics.locations2degree`.

#### Step 4: Calculate Theoretical Arrivals

```python
from obspy.taup import TauPyModel
m = TauPyModel(model="ak135")
arrivals = m.get_ray_paths(...)
arrivals.plot()
```

#### Step 5: Calculate absolute time of the first arrivals at the station

#### Step 6:  Cut to 1 minute before and 5 minutes after the first arrival

#### Step 7: Remove the instrument response

```python
st.attach_response(inv)
st.remove_response(pre_filt=...)
```

![taper](images/cos_taper.png)

## Bonus: Interactive IPython widgets

In [ ]:
from IPython.html.widgets import interact
from obspy.taup import TauPyModel

m = TauPyModel("ak135")

def plot_raypaths(distance, depth, wavetype):
    try:
        plt.close()
    except:
        pass
    if wavetype == "ttall":
        phases = ["ttall"]
    elif wavetype == "diff":
        phases = ["Pdiff", "pPdiff"]
    m.get_ray_paths(distance_in_degree=distance,
                    source_depth_in_km=depth,
                    phase_list=phases).plot();
    
interact(plot_raypaths, distance=[0, 180],
         depth=[0, 700], wavetype=["ttall", "diff"]);